# EP02: 머신러닝용 GPU 클러스터 쉽게 만들기

오프닝: 간단한 본인 소개

Q: 이번에 살펴볼 내용은 어떤 내용인가요?

A: 이번 에피소드에서는 머신 러닝용 GPU 클러스터를 만들고, 그 위에 텐서플로우로 MNIST 트레이닝 프로그램을 구동하는 방법을 AKS 엔진을 활용하여 살펴보려고 합니다.

Q: GPU 자원은 쿠버네티스에서 어떻게 사용할 수 있나요?

A: 아시다시피 GPU 자원은 컴퓨팅 자원과는 달리 기존 컴퓨터에 직접 또는 여러 컴퓨터가 공유해서 연결하는 형태로 나누어 쓰도록 되어있습니다. 그래서 쿠버네티스에서 GPU를 사용하려면 별도의 플러그인을 설치해주어야 합니다.

또한 GPU를 사용할 수 있는 가상 컴퓨터 인스턴스의 종류도 한정되어있는데, N 시리즈의 가상 컴퓨터를 사용해야 합니다. N 시리즈의 가상 컴퓨터는 다른 클래스보다 사양이 더 높으면서도, 고성능의 GPU를 사용할 수 있기 때문에 시간 당 단가가 더 높습니다.

## 애저 로그인 상태 확인

우선 애저 커맨드 라인 도구의 로그인 상태를 먼저 확인합니다. 로그인 상태가 아니면 다른 터미널에서 `az login` 명령을 실행합니다. 설치되어있지 않다면 OS 버전 별로 애저 커맨드 라인 도구를 설치하고 준비합니다.

In [ ]:
az account list

## AKS 엔진 설치 상태 확인

아래 명령어를 실행하여 `aks-engine`이 설치되었는지 확인합니다. 설치되어있지 않다면 OS 버전 별로 각각 설치 가이드를 따라 설치를 진행합니다.

In [ ]:
aks-engine version

## 서비스 주체 생성

쿠버네티스 클러스터를 만든 후에는 클러스터 수준에서 스스로 애저 리소스들을 제어할 수 있어야 하므로, 서비스 주체 계정을 만들어 계속 해당 계정을 사용하도록 설정합니다.

In [ ]:
$SubscriptionData = (az account list -o json) | ConvertFrom-Json -Depth 16
$SubscriptionId = $SubscriptionData[0].id
$SvcPrincipal = (az ad sp create-for-rbac `
  -n="AzurePlaylist" --role="Contributor" `
  --scopes="/subscriptions/$SubscriptionId" `
  -o json) | ConvertFrom-Json -Depth 16

$PrincipalId = $SvcPrincipal.AppId
$PrincipalPwd = $SvcPrincipal.Password

"Your App ID is $PrincipalId"

## SSH 키 준비하기

아래 명령어를 실행하여 SSH 키의 공개 키를 준비합니다. 설치되어있지 않다면 SSH 키 쌍을 만듭니다.

In [ ]:
$PublicKeyValue = Get-Content -Path "$HOME\.ssh\id_rsa.pub"

## API 모델 준비하기

이제 API 모델을 준비할 차례입니다. AKS 엔진은 코드 기반 인프라 관리 방식을 구현하기 위하여 JSON API 모델을 작성하면, 여기에 맞추어 ARM 템플릿을 만들어줍니다. 이것을 애저 명령줄 도구를 통해 배포하면 손쉽게 쿠버네티스 클러스터를 생성할 수 있습니다.

In [ ]:
$ApiModel = @"
{
  'apiVersion': 'vlabs',
  'properties': {
    'orchestratorProfile': {
      'orchestratorType': 'Kubernetes'
    },
    'servicePrincipalProfile': {
      'clientId': '',
      'secret': ''
    },
    'masterProfile': {
      'count': 1,
      'vmSize': 'Standard_D2_v3'
    },
    'agentPoolProfiles': [
      {
        'name': 'agentpool1',
        'count': 3,
        'vmSize': 'Standard_NC6',
        'availabilityProfile': 'AvailabilitySet'
      }
    ],
    'linuxProfile': {
      'adminUsername': 'azureuser',
      'ssh': {
        'publicKeys': [
          {
            'keyData': ''
          }
        ]
      }
    }
  }
}
"@ | ConvertFrom-Json -Depth 16

## API 모델 내용 변경하기

나중에 리눅스 마스터 노드에 접속할 수 있도록 현재 컴퓨터에서 사용하는 SSH 공개 키를 배포할 때 같이 등록합니다. 그리고 서비스 주체를 가리키는 아이디 값과 고유 비밀 키 값을 API 모델에 같이 지정합니다.

In [ ]:
$ApiModel.properties.linuxProfile.ssh.publicKeys[0].keyData = "$PublicKeyValue"
$ApiModel.properties.servicePrincipalProfile.clientId = "$PrincipalId"
$ApiModel.properties.servicePrincipalProfile.secret = "$PrincipalPwd"

In [ ]:
$ApiModel | ConvertTo-Json -Depth 16 | Out-File -Encoding utf8 -Path 'ep02.json' -Force

dir 'ep02*'

## API 모델을 ARM 템플릿으로 변환하고 배포하기

다음의 명령어를 실행하여 API 모델을 ARM 템플릿으로 변환하고 곧바로 애저에 배포하도록 하겠습니다. 이 명령어는 최초에 한 번만 실행하고, 그 이후에는 `ep01_output` 디렉터리 안에 있는 파일들을 이용하여 클러스터를 관리하도록 합니다.

In [ ]:
$DnsPrefix = 'rkttuep02'
$ResourceGroup = 'rkttuep02'
$Location = 'eastus'

In [ ]:
aks-engine deploy --dns-prefix "$DnsPrefix" `
    --resource-group "$ResourceGroup" `
    --location "$Location" `
    --api-model "ep02.json" `
    --auto-suffix `
    --output-directory "ep02_output" `
    --auth-method cli

## kubectl 실행해보기

클러스터가 잘 만들어졌는지 확인해보기 위하여 다음과 같이 `KUBECONFIG` 환경 변수를 설정하여 클러스터에 접근하고, 노드와 구성 상태를 확인합니다.

In [ ]:
$env:KUBECONFIG="ep02_output/kubeconfig/kubeconfig.$Location.json"
kubectl get nodes -o wide

## NVIDIA GPU 플러그인 설치하기

쿠버네티스 클러스터에 배포되는 애플리케이션에서 GPU를 사용할 수 있도록 디바이스 플러그인을 설치해야 합니다.

In [ ]:
kubectl create namespace gpu-resources

$DevicePluginYaml = @'
apiVersion: apps/v1
kind: DaemonSet
metadata:
  name: nvidia-device-plugin-daemonset
  namespace: gpu-resources
spec:
  selector:
    matchLabels:
      name: nvidia-device-plugin-ds
  updateStrategy:
    type: RollingUpdate
  template:
    metadata:
      # Mark this pod as a critical add-on; when enabled, the critical add-on scheduler
      # reserves resources for critical add-on pods so that they can be rescheduled after
      # a failure.  This annotation works in tandem with the toleration below.
      annotations:
        scheduler.alpha.kubernetes.io/critical-pod: ""
      labels:
        name: nvidia-device-plugin-ds
    spec:
      tolerations:
      # Allow this pod to be rescheduled while the node is in "critical add-ons only" mode.
      # This, along with the annotation above marks this pod as a critical add-on.
      - key: CriticalAddonsOnly
        operator: Exists
      - key: nvidia.com/gpu
        operator: Exists
        effect: NoSchedule
      containers:
      - image: mcr.microsoft.com/oss/nvidia/k8s-device-plugin:1.11
        name: nvidia-device-plugin-ctr
        securityContext:
          allowPrivilegeEscalation: false
          capabilities:
            drop: ["ALL"]
        volumeMounts:
          - name: device-plugin
            mountPath: /var/lib/kubelet/device-plugins
      volumes:
        - name: device-plugin
          hostPath:
            path: /var/lib/kubelet/device-plugins
'@

$DevicePluginYaml | kubectl apply -f -

## MNIST 트레이닝 구동하기

배포한 GPU 쿠버네티스 클러스터가 잘 작동하는지 확인해보기 위하여 MNIST 트레이닝 샘플을 구동해보겠습니다.

In [ ]:
$MnistWorkloadYaml = @'
apiVersion: batch/v1
kind: Job
metadata:
  labels:
    app: samples-tf-mnist-demo
  name: samples-tf-mnist-demo
spec:
  template:
    metadata:
      labels:
        app: samples-tf-mnist-demo
    spec:
      containers:
      - name: samples-tf-mnist-demo
        image: mcr.microsoft.com/azuredocs/samples-tf-mnist-demo:gpu
        args: ["--max_steps", "500"]
        imagePullPolicy: IfNotPresent
        resources:
          limits:
           nvidia.com/gpu: 1
      restartPolicy: OnFailure
'@

$MnistWorkloadYaml | kubectl apply -f -

In [ ]:
kubectl get jobs samples-tf-mnist-demo

In [ ]:
$MnistWorkload = (kubectl get pods --selector app=samples-tf-mnist-demo -o json) | ConvertFrom-Json -Depth 16

$PodName = $MnistWorkload.items[0].metadata.name

In [ ]:
kubectl logs $PodName | head -n 10

In [ ]:
kubectl logs --tail=10 $PodName

In [ ]:
kubectl delete jobs samples-tf-mnist-demo

## 리소스 그룹 정리

테스트를 위해 생성한 리소스 그룹을 제거하고 모든 리소스를 일괄 정리합니다.

In [ ]:
az group delete --name $ResourceGroup --yes --no-wait

클로징: 이 동영상을 보고 애저를 써봐야겠다고 생각하셨나요? 애저 계정은 누구나 무료로 만들 수 있습니다. 지금 바로 시작해보세요. (가능하다면 할인/무료 쿠폰을 포함한 가입 링크 제공) 다음 에피소드에서는 클라우드 셸을 사용할 수 있는 다양한 방법을 알아보겠습니다.